# YARN on Stampede

This is an advanced example intended to show the capabilities. To execute this example a Stampede account is required.

# 1. Setting up the enviroment

Before running the following example you will need to set a passwordless ssh connection between your tutorial account and your account on TACC Stampede.

First, login through ssh to the edge node with the tutorial account and set up a public key:
```
ssh <user>@sc15-aws.radical-cybertools.org
ssh-keygen -t rsa
```

Hit Return until you are in the terminal command prompt again.

Then execute the following command
```
cat .ssh/id_rsa.pub | ssh <tacc_username>@stampede.tacc.utexas.edu 'cat >> .ssh/authorized_keys'
```

Verify that everything is set by doing and you can connect without a password request:
```
ssh <tacc_username>@stampede.tacc.utexas.edu
```


# 2. Radical Pilot Setup

In [1]:
import os,sys
import radical.pilot as rp
import ast

os.environ["RADICAL_PILOT_DBURL"]="mongodb://ec2-54-221-194-147.compute-1.amazonaws.com:24242/sc15tut"
os.environ["RADICAL_PILOT_VERBOSE"]="DEBUG"

def print_details(detail_object):
    if type(detail_object)==str:
        detail_object = ast.literal_eval(detail_object)
    for i in detail_object:
        detail_object[i]=str(detail_object[i])
    return pd.DataFrame(detail_object.values(), 
             index=detail_object.keys(), 
             columns=["Value"])

# 3. Create a Radical Pilot Session

In [2]:
session = rp.Session()
print "Session id %s"%session.uid
c = rp.Context('ssh')
c.user_id = ""
session.add_context(c)

Session id rp.session.ip-10-99-194-113.ec2.internal.iparask.016749.0001


# 4. Create Pilot and Unit Managers

In [4]:
print "Initializing Pilot Manager ..."
pmgr = rp.PilotManager(session=session)
print "Initializing Unit Manager ..."
umgr = rp.UnitManager (session=session,
                           scheduler=rp.SCHED_DIRECT_SUBMISSION)

Initializing Pilot Manager ...
Initializing Unit Manager ...


# 5. Submit the pilot to the Pilot and Unit Managers

In [5]:
pdesc = rp.ComputePilotDescription ()
pdesc.resource = "yarn.stampede"  # NOTE: This is a "label", not a hostname
pdesc.runtime  = 60 # minutes
pdesc.cores    = 16
pdesc.cleanup  = False
pdesc.project  = '' #Include the Allocation here
pdesc.queue    = 'development' #You can select a different queue if you want.
# submit the pilot.
print "Submitting Compute Pilot to Pilot Manager ..."
pilot = pmgr.submit_pilots(pdesc)
print "Registering Compute Pilot with Unit Manager ..."
umgr.add_pilots(pilot)

Submitting Compute Pilot to Pilot Manager ...
Registering Compute Pilot with Unit Manager ...


# 6. Submit Compute Units

First create the description of the compute units which define the task to be executed

In [6]:
NUMBER_JOBS  = 16
cudesc_list = []
for i in range(NUMBER_JOBS):
    cudesc = rp.ComputeUnitDescription()
    cudesc.environment = {'CU_NO': i}
    cudesc.executable  = "/bin/echo"
    cudesc.arguments   = ['I am CU number $CU_NO']
    cudesc.cores       = 1
    cudesc_list.append(cudesc)

Submit the created Compute Units to the Unit Manager.

In [7]:

print "Submit Compute Units to Unit Manager ..."
cu_set = umgr.submit_units (cudesc_list)
print "Waiting for CUs to complete ..."
umgr.wait_units()
print "All CUs completed successfully!"

Submit Compute Units to Unit Manager ...
Waiting for CUs to complete ...
All CUs completed successfully!


Printing the output of a Compute Unit

In [8]:
for unit in cu_set:
    print "* CU %s, state %s, exit code: %s, stdout: %s" \
     % (unit.uid, unit.state, unit.exit_code, unit.stdout)

* CU unit.000000, state Done, exit code: 0, stdout: [... CONTENT SHORTENED ...]
pede.tacc.utexas.edu/129.114.90.162:8032 from tg824689 sending #2466
2015-11-10 16:27:25 DEBUG Client:1089 - IPC Client (1726925370) connection to c557-402.stampede.tacc.utexas.edu/129.114.90.162:8032 from tg824689 got value #2466
2015-11-10 16:27:25 DEBUG ProtobufRpcEngine:250 - Call: getApplicationReport took 0ms
2015-11-10 16:27:25 DEBUG Client:1032 - IPC Client (1726925370) connection to c557-402.stampede.tacc.utexas.edu/129.114.90.162:8032 from tg824689 sending #2467
2015-11-10 16:27:25 DEBUG Client:1089 - IPC Client (1726925370) connection to c557-402.stampede.tacc.utexas.edu/129.114.90.162:8032 from tg824689 got value #2467
2015-11-10 16:27:25 DEBUG ProtobufRpcEngine:250 - Call: getApplicationReport took 0ms
2015-11-10 16:27:25 INFO  Client:792 - Application has completed successfully. Breaking monitoring loop
2015-11-10 16:27:25 INFO  Client:240 - Application completed successfully
2015-11-10 16:27:

# 7. Always clean up the session

In [9]:
session.close ()
del session